In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
# import keras.utils as ku 
import tensorflow.keras.utils as k
# set seeds for reproducability
import tensorflow
from numpy.random import seed
# tensorflow.random.set_seed(2)
# seed(1)
import tensorflow as tf

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
f = open("harry.txt", "r")
x = f.read()

In [3]:
# period_separated_sentence = x.split('.')


In [4]:
import re
y = ''
cleaned = re.findall(r'[^!?",\[\]\n\s\d%]+', x)
for i in cleaned:

    y+=i
    y = y+' '

In [5]:
period_separated_sentence = y.split('.')

In [6]:
period_separated_sentence[:5]

['THE BOY WHO LIVED Mr',
 ' and Mrs',
 ' Dursley of number four Privet Drive were proud to say that they were perfectly normal thank you very much',
 ' They were the last people you’d expect to be involved in anything strange or mysterious because they just didn’t hold with such nonsense',
 ' Mr']

In [7]:

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(period_separated_sentence)
inp_sequences[:10]

[[1, 245],
 [1, 245, 54],
 [1, 245, 54, 1618],
 [1, 245, 54, 1618, 91],
 [3, 207],
 [906, 5],
 [906, 5, 810],
 [906, 5, 810, 437],
 [906, 5, 810, 437, 1161],
 [906, 5, 810, 437, 1161, 973]]

In [8]:
tokenizer

In [9]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = k.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [10]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 194, 10)           164560    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               44400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 16456)             1662056   
Total params: 1,871,016
Trainable params: 1,871,016
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(predictors, label, epochs=100, verbose=5)
model.save('harry_model_100.h5')

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [11]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()



In [13]:
import time 
string_input = "he growled at the Dursleys, 'that this boy — this boy! — knows nothin'"
start = time.time()
print (generate_text(string_input, 10, model, max_sequence_len))
end = time.time()
print((end - start)*1000, ' miliseconds')

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
He Growled At The Dursleys, 'That This Boy — This Boy! — Knows Nothin' That ” Said Harry Looking At The Door And The
692.5339698791504  miliseconds


In [17]:
max_sequence_len

195

In [22]:
#saving the model 
model.save('harry_model.h5')

In [8]:
#loading the model
model1 = tf.keras.models.load_model('harry_model_100.h5')

In [9]:
max_sequence_len = 195
tokenizer = Tokenizer()
tokenizer.fit_on_texts(period_separated_sentence)

In [14]:
# Printing from the loaded model 
print (generate_text('The chamber of', 15, model1, max_sequence_len))

The Chamber Of Secrets And Was The Only Person In The World — ” Page Harry Potter And


In [28]:
input_string = 'Snape'

In [29]:
import time 
start = time.time()
print (generate_text(input_string, 15, model1, max_sequence_len))
print(time.time() - start)

Snape Was Standing In The Doorway And Saw A Few Of His Wand And Then He
0.3387877941131592


In [30]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            0.13.0
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
anyio                              2.2.0
appdirs                            1.4.4
applaunchservices                  0.2.1
appnope                            0.1.2
appscript                          1.1.2
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.5
astropy                            4.2.1
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.6
Babel                              2.9.0
backcall                           0.2.